In [90]:
from pyspark.sql import SparkSession
import json

In [163]:
appName = "Nike Spark test"
master = 'local'
spark = SparkSession.builder \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

In [164]:
calender = spark.read.format('csv') \
                .option('header',True) \
                .option('multiLine', True) \
                .load('./calendar.csv')

consumption = spark.read.format('csv') \
                .option('header',True) \
                .option('multiLine', True) \
                .load('./consumption.csv')

product = spark.read.format('csv') \
                .option('header',True) \
                .option('multiLine', True) \
                .load('./product.csv')

sales = spark.read.format('csv') \
                .option('header',True) \
                .option('multiLine', True) \
                .load('./sales.csv')

store = spark.read.format('csv') \
                .option('header',True) \
                .option('multiLine', True) \
                .load('./store.csv')

In [165]:
calender = calender.selectExpr("datekey as dateId", "datecalendarday as datecalendarday",
                               "datecalendaryear as datecalendaryear", "weeknumberofseason as weeknumberofseason")

In [166]:
product_sales = product.join(sales, on='productid')

In [167]:
data = store.join(product_sales, on='storeId')

In [168]:
data = data.join(calender, on='dateId', how='left')

In [170]:
data_json = data.toJSON().collect()
data_json = [json.loads(string_data) for string_data in data_json]
def create_week_format(_data):
    print(_data.keys())
    week_target = _data['weeknumberofseason']
    week_dict = {}
    for weekly_year in range(1, 53):
        if weekly_year == int(week_target):
            week_dict[f'W{weekly_year}'] = _data['netSales'] # should be multiplied by salesUnits?
        else:
            week_dict[f'W{weekly_year}'] = 0
    return week_dict
print(data_json)
"""
for element in data_json:
    print(element)
    element['weekly_yearly sales'] = create_week_format(create_week_format(element))
#def create_data_rows(data, comment='test'):
    #row_id = f'{saleId}_{storeid}'"""

[{'dateId': '4965', 'storeid': '409', 'channel': 'Digital', 'country': 'INDIA', 'productid': '00567228914507', 'division': 'APPAREL', 'gender': 'KIDS', 'category': 'CRICKET', 'saleId': '1', 'netSales': '300.24', 'salesUnits': '5', 'datecalendarday': '1', 'datecalendaryear': '2018', 'weeknumberofseason': '2'}, {'dateId': '4965', 'storeid': '409', 'channel': 'Digital', 'country': 'INDIA', 'productid': '00567228914507', 'division': 'APPAREL', 'gender': 'KIDS', 'category': 'CRICKET', 'saleId': '2', 'netSales': '300.24', 'salesUnits': '5', 'datecalendarday': '1', 'datecalendaryear': '2018', 'weeknumberofseason': '2'}, {'dateId': '4965', 'storeid': '409', 'channel': 'Digital', 'country': 'INDIA', 'productid': '00567228914507', 'division': 'APPAREL', 'gender': 'KIDS', 'category': 'CRICKET', 'saleId': '3', 'netSales': '300.24', 'salesUnits': '5', 'datecalendarday': '1', 'datecalendaryear': '2018', 'weeknumberofseason': '2'}, {'dateId': '4965', 'storeid': '409', 'channel': 'Digital', 'country':

"\nfor element in data_json:\n    print(element)\n    element['weekly_yearly sales'] = create_week_format(create_week_format(element))\n#def create_data_rows(data, comment='test'):\n    #row_id = f'{saleId}_{storeid}'"

In [151]:
element

{'dateId': '4965',
 'storeid': '409',
 'channel': 'Digital',
 'country': 'INDIA',
 'productid': '00567228914507',
 'division': 'APPAREL',
 'gender': 'KIDS',
 'category': 'CRICKET',
 'saleId': '1',
 'netSales': '300.24',
 'salesUnits': '5',
 'datecalendarday': '1',
 'datecalendaryear': '2018',
 'weeknumberofseason': '2'}

In [158]:
consumption.show(23432)

+--------------------+
|                {   |
+--------------------+
|  "_comment": "Un...|
|  "uniqueKey": "R...|
|  "division": "DIV1"|
|    "gender": "KIDS"|
|  "category": "EQ...|
|  "channel": "DIG...|
|      "year": "RY18”|
|      "dataRows": [ |
|               {    |
|      "_comment":...|
|     "rowId": "Ne...|
|        "dataRow": {|
|          "W1": 5000|
|             "W2": 0|
|             "W3": 0|
|             "W4": 0|
|             "W5": 0|
|             "W6": 0|
|             "W7": 0|
|             "W8": 0|
|             "W9": 0|
|            "W10": 0|
|            "W11": 0|
|            "W12": 0|
|            "W13": 0|
|         "W14": 2000|
|            "W15": 0|
|            "W16": 0|
|         "W17": 3000|
|            "W18": 0|
|            "W19": 0|
|            "W20": 0|
|            "W21": 0|
|            "W22": 0|
|            "W23": 0|
|            "W24": 0|
|            "W25": 0|
|            "W26": 0|
|            "W27": 0|
|            "W28": 0|
|          

In [160]:
data.show()

+------+-------+-------+------------+--------------+--------+------+--------+------+--------+----------+---------------+----------------+------------------+
|dateId|storeid|channel|     country|     productid|division|gender|category|saleId|netSales|salesUnits|datecalendarday|datecalendaryear|weeknumberofseason|
+------+-------+-------+------------+--------------+--------+------+--------+------+--------+----------+---------------+----------------+------------------+
|  4965|    409|Digital|       INDIA|00567228914507| APPAREL|  KIDS| CRICKET|     1|  300.24|         5|              1|            2018|                 2|
|  4965|    409|Digital|       INDIA|00567228914507| APPAREL|  KIDS| CRICKET|     2|  300.24|         5|              1|            2018|                 2|
|  4965|    409|Digital|       INDIA|00567228914507| APPAREL|  KIDS| CRICKET|     3|  300.24|         5|              1|            2018|                 2|
|  4965|    409|Digital|       INDIA|00567228914507| APPAR